In [4]:
import os
import tweepy as tw
import pandas as pd
import time

In [3]:
time.sleep(10)

In [5]:
# data paths
#   for the record: these were scraped on january 26th 2021
player_twitters = 'data/player-twitters.csv'
def draft_path(year):
    return 'data/drafts/'+str(year)+'-draft.csv'
def get_draft_df(year_from,year_to):
    df = pd.read_csv(draft_path(year_from))
    df['Year'] = [year_from]*len(df)
    for yr in range(year_from+1,year_to+1):
        new_df = pd.read_csv(draft_path(yr))
        new_df['Year'] = [yr]*len(new_df)
        df = pd.concat([df,new_df])
    return df
def add_twitter_data(handles, draft_df):
    # build lookup table for twitter handles
    hndls = {}
    for i in range(len(handles['Player'])):
        name = handles['Player'][i].split('\\')[0]
        hndls[name] = handles['Twitter'][i]
    draft_twitters = []
    for player in draft_df['Player']:
        if player in hndls:
            draft_twitters.append(hndls[player])
        else:
            draft_twitters.append('')
    draft_df['twitter_handle'] = draft_twitters
# dataframes
handles = pd.read_csv(player_twitters)
draft_df = get_draft_df(2010,2020)
add_twitter_data(handles,draft_df)

In [6]:
draft_df.to_csv('data/drafts/all_drafts.csv',index=False)

In [7]:
draft_df.sample(5)

,Rk,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,MP.1,PTS.1,TRB.1,AST.1,WS,WS/48,BPM,VORP,Year,twitter_handle
29,30,30,BOS,Desmond Bane,TCU,1.0,13.0,285.0,112.0,36.0,...,21.9,8.6,2.8,1.1,0.5,0.080,-3.0,-0.1,2020,DBane0625
57,58,58,MIN,Robbie Hummel,Purdue,2.0,98.0,1397.0,379.0,266.0,...,14.3,3.9,2.7,0.5,1.9,0.066,-3.1,-0.4,2012,RobbieHummel
15,16,16,MIN,Luke Babbitt,Nevada,8.0,381.0,5332.0,1818.0,846.0,...,14.0,4.8,2.2,0.6,7.2,0.065,-2.1,-0.1,2010,Realbabbitt08
26,27,27,UTA,Udoka Azubuike,Kansas,1.0,9.0,44.0,12.0,11.0,...,4.9,1.3,1.2,0.0,0.1,0.107,-5.0,0.0,2020,TimUdoka
47,48,48,GSW,Nico Mannion,Arizona,1.0,4.0,28.0,9.0,3.0,...,7.0,2.3,0.8,2.5,0.1,0.096,0.6,0.0,2020,niccolomannion


In [4]:
len(draft_df['Rk'])

660

In [5]:
# it turns out this is the most amount of tweets we'll be able to get
#   from each player without going over the limit - 500,000
# 1100 tweets per player
# we'll also exclude players drafted 2019 and 2020, since in a lot 
#   of ways the jury is still out on them.
draft_df_subset = draft_df[(draft_df['Year'] <= 2018) & (draft_df['Twitter'] != '')]

In [6]:
# twitter keys
API_key = 'oixmKPW8n8t2lc36fPdl4mSAF'
API_secret_key = '32UMo5DcM8dMrgBIzYn46TAHw4PWdg73FLx5sCLmBRl3H10V8K'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAJ5TMAEAAAAApFSd9mnuFxae5fcUH3D%2FJKDYNi0%3DBjmfMoXjVlBjPTasAomlUd4AqlctFmk7GLml80DObF1yl4Lgsf'

auth = tw.AppAuthHandler(API_key, API_secret_key)
api = tw.API(auth)

In [7]:
def get_player_tweets(hndl, num_tweets):
    '''
    hndl: string, representing twitter handle
    returns a pandas series with the following variables:
        - handle
        - cleaned date/time
        - cleaned tweet text
    '''
    player_tweets = pd.DataFrame(columns = ['handle', 'time', 'text', 'mentions'])
    try:
        for i, tweet in enumerate(tw.Cursor(api.user_timeline, id = hndl).items(num_tweets)):
            row = [hndl, tweet.created_at,tweet.text, ','.join([m['screen_name'] for m in tweet.entities['user_mentions']])]
            player_tweets.loc[i] = row
    except:
        print('Skipping user: '+hndl)
    return player_tweets

In [8]:
def get_all_tweets(draft_df, num_tweets_per, old_tweets, last_user_captured):
    tweets = pd.DataFrame(columns = ['handle', 'time', 'text', 'mentions'])
    for handle in draft_df['Twitter']:
        if (handle == last_user_captured) or (handle not in set(old_tweets['handle'])):
            if handle != '':
                try:
                    player_tweets = get_player_tweets(handle,num_tweets_per)
                    tweets = pd.concat([tweets,player_tweets])
                except:
                    pass
    return tweets

In [5]:
tweets = pd.read_csv('data/player_tweets.csv')

In [6]:
len(tweets['handle'])

265021

In [9]:
#new_tweets = get_all_tweets(draft_df_subset, 800, tweets, '1jordanbell')

In [10]:
#tweets = pd.concat([tweets,new_tweets])

In [11]:
tweets.drop_duplicates(inplace=True)
tweets.to_csv('data/player_tweets.csv', index=False)

In [12]:
tweets.tail()

,handle,time,text,mentions
265215,Kostas_ante13,2014-02-10 06:12:06,RT @G_ante34: John Newman- Love me again #Gian...,G_Ante34
265216,Kostas_ante13,2014-02-08 00:50:24,Tomorrow I got my first practice in Milwaukee ...,NaN
265217,Kostas_ante13,2014-02-07 01:48:28,Milwaukee is so cold 💨❄️❄️⛄️..But is very nice!!,NaN
265218,Kostas_ante13,2014-02-06 23:44:45,RT @G_ante34: We are gonna be the first family...,G_Ante34
265219,Kostas_ante13,2014-02-06 23:43:43,RT @G_ante34: My brothers first smoothies..!! ...,G_Ante34


In [13]:
print('# Players Scraped (Total): '+str(len(set(tweets['handle']))))
print('# Tweets Scraped (Total): '+str(len(tweets['handle'])))

# Players Scraped (Total): 433
# Tweets Scraped (Total): 265021


In [14]:
# DELETEME: Isaac
isaac = get_player_tweets('isaackendell', 500)
import preprocessor as p

In [16]:
isaac_words = p.clean(' '.join(isaac['text']))

In [ ]:
tweets.head()

In [9]:
# TODO:
#  create a year column in the tweets df that stores the season that
#     the tweet came from
#  join player name to the tweets df

In [11]:
mrgn = get_player_tweets('morganlhodson', 620)

In [12]:
mrgn.head(10)

,handle,time,text,mentions
0,morganlhodson,2021-01-23 07:52:11,Anyway long story short I forgot that knives a...,
1,morganlhodson,2021-01-23 07:51:19,Okay so I’ve been using awful knives for years...,
2,morganlhodson,2021-01-13 04:24:19,I’m pretty sure it’s a bad omen when your bran...,
3,morganlhodson,2020-12-13 03:35:41,"The more I look at my grades, the more serious...",
4,morganlhodson,2020-11-12 03:25:28,I think it’s never. The homework is never actu...,
5,morganlhodson,2020-11-12 03:18:06,I think that made it worse. Okay I miss homewo...,
6,morganlhodson,2020-11-12 03:16:44,That sounded nerdy. I’d like to clarify that I...,
7,morganlhodson,2020-11-12 03:15:35,I miss math,
8,morganlhodson,2020-11-04 17:08:26,RT @lici_lopez: COME ON WE CAN DO THIS ITS SO ...,lici_lopez
9,morganlhodson,2020-11-02 05:20:29,I have baked 3 cakes and missed so many assign...,


In [13]:
mrgn = ' '.join(mrgn['text']).lower()

In [14]:
f = open('mrgn_texts.txt','w')
f.write(mrgn)
f.close()